In [1]:
import random
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# !pip3 install transformers
from transformers import pipeline
import torch
device = -1
if torch.cuda.is_available():
  device = 0

question_answering = pipeline("question-answering", device=device)
from google.colab import drive
drive.mount('/content/gdrive/')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Mounted at /content/gdrive/


In [2]:
CORPUS_PATH=\
'gdrive/MyDrive/Colab Notebooks/MSDS453/FinalProject/parks_clean.csv'
national_parks_corpus = pd.read_csv(CORPUS_PATH)
national_parks_corpus.head(20)

,park,text,char_length,token_lengths
0,Acadia National Park,Acadia National Park is an American national p...,46957,7341
1,Arches National Park,Arches National Park is a national park in eas...,16175,2536
2,Badlands National Park,Badlands National Park (Lakota: Makȟóšiča) is ...,19424,3049
3,Big Bend National Park,Big Bend National Park is an American national...,23315,3722
4,Biscayne National Park,Biscayne National Park is an American national...,49014,7817
5,Black Canyon of the Gunnison National Park,Black Canyon of the Gunnison National Park is ...,17075,2897
6,Bryce Canyon National Park,Bryce Canyon National Park () is an American n...,18531,3001
7,Canyonlands National Park,Canyonlands National Park is an American natio...,18483,2951
8,Capitol Reef National Park,Capitol Reef National Park is an American nati...,21180,3315
9,Carlsbad Caverns National Park,Carlsbad Caverns National Park is an American ...,17978,2912


In [3]:
ALL_DOCUMENT_CONTEXT = national_parks_corpus['text'].str.cat(sep='').lower()

In [4]:
def answer_question(question, park=''):
  if park:
    if park in national_parks_corpus.park.to_list():
      context = national_parks_corpus.loc[national_parks_corpus['park'] == park, 'text'].iloc[0].lower()
    else:
      return "Sorry, I don't know that park."
  else:
    context = ALL_DOCUMENT_CONTEXT
  result = question_answering(question=question.lower(), context=context)
  return result['answer'], result['score']

In [5]:
start = time.time()
result = answer_question('What types of animals will I see at Denali?', 'Denali National Park and Preserve')
print(result)
print(f"Response time: {round(time.time()-start, 4)} seconds")

('mammals', 0.6492246985435486)
Response time: 3.8806 seconds


In [10]:
def run_chatbot():
  times = []
  print("Hi! Welcome to the NPS Chatbot! I can answer questions about National Parks!\nIf you would like to end this session, type 'exit'.")
  flag = True
  while flag:
    prompt = input("USER: ")
    if prompt.lower() == 'exit':
      print("Goodbye!")
      flag=False
      if times:
        print(f"Average response time: {round(np.mean(times),2)} seconds")
      break
    start = time.time()
    response = answer_question(prompt)[0]
    end = time.time()
    times.append(end-start)
    print(f"NPS CHATBOT: {response}")

In [9]:
run_chatbot()

Hi! Welcome to the NPS Chatbot! I can answer questions about National Parks!
If you would like to end this session, type 'exit'.
USER: When was Yellowstone National Park established?
NPS CHATBOT: 1947
USER: What factors make the Grand Canyon unique?
NPS CHATBOT: increased visitation and traffic congestion
USER: What activities are available in Shenandoah National Park?
NPS CHATBOT: farming or mining
USER: Which national parks are located in California?
NPS CHATBOT: everglades
USER: Explain the potential effects of climate change on Glacier National Park's glaciers and ecosystems. How can the park adapt to these changes?
NPS CHATBOT: negatively affected food supplies throughout the continent
USER: exit
Goodbye!
Average response time: 26.746 seconds
Average Response time: 26.75 seconds
